<center class="mytitle">
**Python avancé**
</center>

# Les tests

<center>
<span>**Loic Gouarin**</span>
</center>
<center>
<span>21 et 22 novembre 2017</span>
</center>

### Pourquoi écrire des tests ?

Lorsque l'on écrit un programme, il est généralement constitué de plusieurs fonctions que l'on assemble afin de décrire notre algorithme permettant de nous donner la réponse à notre problème. Un programme n'est pas forcément un développement sur un temps court. On voit beaucoup de librairies scientifiques qui ont plus de dix ans. Les fonctions peuvent donc être écrites à différents moments avec des échelles de temps bien différentes. On peut par exemple ajouter une fonctionnalité à un bout de code plusieurs années après en avoir écrit le coeur. Si il est primordial d'écrire de la documentation pour comprendre ce qui est fait, il est également judicieux d'écrire des tests pour s'assurer du bon fonctionnement de notre programme.

Il faut noter que certains types de développement logiciel s'appuient sur les tests ([Test Driven Development](http://fr.wikipedia.org/wiki/Test_Driven_Development)).

## Les types de tests

On peut citer trois types de tests primordiaux permettant de s'assurer au mieux de l'absence de bugs dans notre programme. Un programme n'est jamais à 100% sûr.

- les **tests unitaires** permettent de tester des fonctions ou des méthodes. 

- les **tests d'intégration** permettent de tester les interactions entre un petit nombre d'unités de programme.

- les **tests du système complet** permettent de tester le programme dans sa globalité.

Les tests sont donc écrits à des stades différents du développement mais ont chacun leur importance. 

Un seul de ces trois types de tests ne suffit pas pour tester l'intégrité du programme. 

Les tests unitaires et les tests d'intégration sont généralement testés avec les mêmes outils. 

Pour le dernier type de tests, on prendra des exemples concrets d'exécution et on testera la sortie avec une solution certifiée.

## Notre cas d'étude

Nous allons calculer les coefficients de la suite de Fibonacci en utilisant les coefficients binomiaux. Les Coefficients binomiaux se calculent à partir de la formule suivante

$$
\left(
\begin{array}{c}
n \\
k
\end{array}
\right)=C_n^k=\frac{n!}{k!(n-k)!} \; \text{pour} \; k=0,\cdots,n.
$$

On en déduit alors le calcul des coefficients de la suite de Fibonacci par la formule suivante

$$
\sum_{k=0}^n
\left(
\begin{array}{c}
n-k \\
k
\end{array} 
\right)
= F(n+1).
$$

Voici un exemple de code Python implantant cette formule

In [14]:
%%file examples/tests/fibonacci.py
import numpy as np

def factorielle(n):
    """
    calcul de n!
    
    >>> factorielle(0)
    1
    >>> factorielle(5)
    120
    
    """
    if n==1 or n==0:
        return 1
    else:
        return n*factorielle(n-1)

def somme(deb, fin, f, fargs=()):
    """
    calcul de 
    
    $$
    \sum_{k=deb}^fin f(k, *fargs)
    $$
    
    test d'une suite arithmetique
    >>> somme(0, 10, lambda k:k)
    55.0
    
    test d'une suite geometrique
    >>> somme(1, 8, lambda k: 2**k)
    510.0

    """
    
    som = 0.
    for k in range(deb, fin + 1):
        som += f(k, *fargs)
    return som
    
def coef_binomial(n, k):
    """
    calcul de $C_n^k$
    
    >>> coef_binomial(4, 2)
    6
    
    """
    if k > n or k < 0:
        return 0.
    return factorielle(n)//(factorielle(k)*factorielle(n-k))

def fibonacci(n):
    """
    Renvoie la liste des n premiers termes de la suite de Fibonacci
    
    >>> fibonacci(10)
    [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
    
    """
    def g(k, n):
        return coef_binomial(n - k, k)
    
    fibo = []
    for i in range(n):
        fibo.append(int(somme(0, i, g, fargs=(i,))))
    
    return fibo

if __name__ == '__main__':
    import doctest
    doctest.testmod(verbose=True)

Overwriting examples/tests/fibonacci.py


On souhaite faire les tests suivants

* **tests unitaires**: tester si les fonctions factorielle et somme fonctionnent correctement.
* **tests d'intégration**: tester si les fonctions factorielle et somme fonctionnent correctement ensemble, tester si la fonction coef_binomial fonctionne correctement.
* **tests du système complet**: tester si la fonction fibonacci donne le bon résultat.

## Les outils de tests en Python

Il existe différents outils en Python permettant de réaliser des tests ([https://wiki.python.org/moin/PythonTestingToolsTaxonomy](https://wiki.python.org/moin/PythonTestingToolsTaxonomy)). 

Nous nous intéresserons ici à trois d'entre eux

- doctest
- unittest
- nose
- pytest

## doctest

doctest permet de faire des tests basiques en s'appuyant sur les docstrings. Celles-ci permettent d'écrire de la documentation très facilement de nos fonctions ou de nos classes Python. Elles se placent tout de suite après une méthode, une fonction, une classe. On rappelle ici brièvement le principe en écrivant une documentation pour une fonction.

`doctest` effectue

* une recherche dans les sources des bouts de texte qui ressemblent à une session interactive Python,
* une recherche dans des fichiers textes des bouts de texte qui ressemblent à une session interactive Python,
* une exécution de ces bouts de session pour voir si le résultat est conforme.

Les sessions interactives sont représentées par le symbole **>>>**.

Pour l'utiliser, il suffit d'importer le module `doctest` et d'appeler `testmod` si on veut tester l'ensemble d'un module comme dans notre exemple `fibonacci.py`.

In [15]:
! python examples/tests/fibonacci.py

Trying:
    coef_binomial(4, 2)
Expecting:
    6
ok
Trying:
    factorielle(0)
Expecting:
    1
ok
Trying:
    factorielle(5)
Expecting:
    120
ok
Trying:
    fibonacci(10)
Expecting:
    [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
ok
Trying:
    somme(0, 10, lambda k:k)
Expecting:
    55.0
ok
Trying:
    somme(1, 8, lambda k: 2**k)
Expecting:
    510.0
ok
1 items had no tests:
    __main__
4 items passed all tests:
   1 tests in __main__.coef_binomial
   2 tests in __main__.factorielle
   1 tests in __main__.fibonacci
   2 tests in __main__.somme
6 tests in 5 items.
6 passed and 0 failed.
Test passed.


Si `doctest` est très simple d'utilisation, on se rend bien compte qu'il est assez limité et qu'il ne permet pas de faire des tests très élaborés.

## unittest

Ce module est également appelé PyUnit et reprend l'esprit de JUnit qui permet de faire des tests en java. 

Il supporte

* les tests automatiques,
* les fonctions d'initialisation et de finalisation pour chaque test,
* l'aggrégation des tests,
* l'indépendance des tests dans le rapport final.

Pour écrire des tests, il faut respecter certaines règles

* Les tests doivent faire partie d'une classe héritée de la classe unittest.TestCase.
* Les noms des méthodes de cette classe doivent avoir le prefixe test pour être considérés comme tests.
* Les tests sont exécutés par ordre alphabétique.
* La fonction exécutée avant chaque test doit avoir le nom setUp.
* La fonction exécutée après chaque test doit avoir le nom tearDown.

In [16]:
%%file examples/tests/test_fibo.py
import unittest
import sys
sys.path.append("./examples/tests")
from fibonacci import *

class TestFibo(unittest.TestCase):
    def test_factorielle_0(self):
        self.assertEqual(factorielle(0), 1)

    def test_factorielle_5(self):
        self.assertEqual(factorielle(5), 120)
        
    def test_somme(self):
        self.assertEqual(somme(0, 10, lambda k:k), 55)

    def test_coef_binomial(self):
        self.assertEqual(coef_binomial(4, 2), 6)
        
    def test_fibo(self):
        self.assertEqual(fibonacci(10), [1, 1, 2, 3, 5, 8, 13, 21, 34, 55])
        
if __name__ == '__main__':
    unittest.main()

Overwriting examples/tests/test_fibo.py


In [17]:
!python examples/tests/test_fibo.py

.....
----------------------------------------------------------------------
Ran 5 tests in 0.001s

OK


## pytest

- très simple à utiliser
- multi plateforme
- comprend `doctest` et `unittest`
- modulaire
- plein de plugins sont disponibles (par exemple [pep8](https://pypi.python.org/pypi/pytest-pep8))

In [24]:
! pytest -v --doctest-module examples/tests

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 9 items                                                               

examples/tests/fibonacci.py::fibonacci.coef_binomial PASSED
examples/tests/fibonacci.py::fibonacci.factorielle PASSED
examples/tests/fibonacci.py::fibonacci.fibonacci PASSED
examples/tests/fibonacci.py::fibonacci.somme PASSED
examples/tests/test_fibo.py::TestFibo::test_coef_binomial PASSED
examples/tests/test_fibo.py::TestFibo::test_factorielle_0 PASSED
examples/tests/test_fibo.py::TestFibo::test_factorielle_5 PASSED
examples/tests/test_fibo.py::TestFibo::test_fibo PASSED
examples/tests/test_fibo.py::TestFibo::test_somme PASSED

=========================== 9 passed in 0.11 seconds ===========================


### assert

Toutes les comparaisons dans `pytest` sont basées sur `assert`.

In [29]:
%%file examples/tests/test_fibo.py

import sys
sys.path.append("./examples/tests")
from fibonacci import *

def test_factorielle_0():
    assert factorielle(0) == 1
    
def test_factorielle_5():
    assert factorielle(5) == 120
        
def test_somme():
    assert somme(0, 10, lambda k:k) == 55

def test_coef_binomial():
    assert coef_binomial(4, 2) == 6
        
def test_fibo():
    assert fibonacci(10) == [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

Overwriting examples/tests/test_fibo.py


In [30]:
! pytest examples/tests/test_fibo.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 5 items                                                               

examples/tests/test_fibo.py .....

=========================== 5 passed in 0.09 seconds ===========================


### `skip` et `skipif`

In [5]:
%%file examples/tests/test_skip.py

import sys
import pytest

@pytest.mark.skip(reason="doesn't work !!")
def test_skip():
    assert True
    
@pytest.mark.skipif(sys.version_info < (3, 6), reason="Python version too old")
def test_skipif():
    assert True

Overwriting examples/tests/test_skip.py


In [7]:
! pytest -v examples/tests/test_skip.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 2 items                                                               

examples/tests/test_skip.py::test_skip SKIPPED
examples/tests/test_skip.py::test_skipif PASSED

===================== 1 passed, 1 skipped in 0.00 seconds ======================


In [18]:
! pytest -v -rxs examples/tests/test_skip.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 2 items                                                               

examples/tests/test_skip.py::test_skip SKIPPED
examples/tests/test_skip.py::test_skipif PASSED
=========================== short test summary info ============================
SKIP [1] examples/tests/test_skip.py:5: doesn't work !!

===================== 1 passed, 1 skipped in 0.00 seconds ======================


### Ajouter une marque

In [13]:
%%file examples/tests/test_mark.py

import pytest

@pytest.mark.slow
def test_slow():
    assert True
    
def test_not_slow():
    assert True

Overwriting examples/tests/test_mark.py


In [14]:
! pytest -v examples/tests/test_mark.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 2 items                                                               

examples/tests/test_mark.py::test_slow PASSED
examples/tests/test_mark.py::test_not_slow PASSED

=========================== 2 passed in 0.01 seconds ===========================


In [15]:
! pytest -v -m slow examples/tests/test_mark.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 2 items                                                               

examples/tests/test_mark.py::test_slow PASSED

============================== 1 tests deselected ==============================
==================== 1 passed, 1 deselected in 0.00 seconds ====================


In [16]:
! pytest -v -m "not slow" examples/tests/test_mark.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 2 items                                                               

examples/tests/test_mark.py::test_not_slow PASSED

============================== 1 tests deselected ==============================
==================== 1 passed, 1 deselected in 0.00 seconds ====================


### Capture de la sortie

In [19]:
%%file examples/tests/test_capture.py

def test_capture():
    print("coucou")
    assert True

Writing examples/tests/test_capture.py


In [20]:
! pytest -v examples/tests/test_capture.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 1 item                                                                

examples/tests/test_capture.py::test_capture PASSED

=========================== 1 passed in 0.00 seconds ===========================


In [21]:
! pytest -v -s examples/tests/test_capture.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 1 item                                                                

examples/tests/test_capture.py::test_capture coucou
PASSED

=========================== 1 passed in 0.00 seconds ===========================


### Exécuter les tests par mots clés

In [22]:
%%file examples/tests/test_key.py

def test_foo_1():
    assert True
    
def test_foo_2():
    assert True
    
def test_bar_1():
    assert True
    
def test_bar_2():
    assert True

def test_bar_3():
    assert True
    

Writing examples/tests/test_key.py


In [28]:
! pytest -v -k foo examples/tests/test_key.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 5 items                                                               

examples/tests/test_key.py::test_foo_1 PASSED
examples/tests/test_key.py::test_foo_2 PASSED

============================== 3 tests deselected ==============================
==================== 2 passed, 3 deselected in 0.01 seconds ====================


In [30]:
! pytest -v -k bar examples/tests/test_key.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 5 items                                                               

examples/tests/test_key.py::test_bar_1 PASSED
examples/tests/test_key.py::test_bar_2 PASSED
examples/tests/test_key.py::test_bar_3 PASSED

============================== 2 tests deselected ==============================
==================== 3 passed, 2 deselected in 0.01 seconds ====================


In [31]:
! pytest -v -k "not bar" examples/tests/test_key.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 5 items                                                               

examples/tests/test_key.py::test_foo_1 PASSED
examples/tests/test_key.py::test_foo_2 PASSED

============================== 3 tests deselected ==============================
==================== 2 passed, 3 deselected in 0.01 seconds ====================


### `fixture`

- Permet de spécifier plus facilement ce qu'il faut faire avant et après un test.
- Peux s'appliquer à une fonction, une classe, un module ou tout le projet.
- Une `fixture` peut appeler une autre `fixture`.
- Une `fixture` est appelée par son nom par le test qui en a besoin.

In [26]:
%%file examples/tests/test_fixture_1.py

import pytest

@pytest.fixture()
def tmpfile():
    with open("tmp_fixture.txt", "w") as f:
        yield f

def test_file(tmpfile):
    tmpfile.write("temporary file : " + tmpfile.name)
    assert True

Overwriting examples/tests/test_fixture_1.py


In [27]:
! pytest -v examples/tests/test_fixture_1.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pep8-1.0.6
collected 1 item                                                                

examples/tests/test_fixture_1.py::test_file PASSED

=========================== 1 passed in 0.01 seconds ===========================


In [28]:
! cat tmp_fixture.txt

temporary file : tmp_fixture.txt

### `parametrize`

Il est également possible de définir un ensemble de paramètres à tester.

In [5]:
%%file examples/tests/test_parametrize_1.py

import sys
sys.path.append("./examples/tests")
import pytest

from fibonacci import *

@pytest.mark.parametrize('fact_number, expected', [
    (0, 1),
    (1, 1),
    (2, 2),
    (3, 6),
    (4, 24),
    (5, 120)
])
def test_methods(fact_number, expected):
    assert factorielle(fact_number) == expected

Writing examples/tests/test_parametrize_1.py


In [6]:
! pytest -v examples/tests/test_parametrize_1.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pylint-0.7.1, pep8-1.0.6, cov-2.5.1
collected 6 items                                                               

examples/tests/test_parametrize_1.py::test_methods[0-1] PASSED
examples/tests/test_parametrize_1.py::test_methods[1-1] PASSED
examples/tests/test_parametrize_1.py::test_methods[2-2] PASSED
examples/tests/test_parametrize_1.py::test_methods[3-6] PASSED
examples/tests/test_parametrize_1.py::test_methods[4-24] PASSED
examples/tests/test_parametrize_1.py::test_methods[5-120] PASSED

=========================== 6 passed in 0.08 seconds ===========================


In [7]:
%%file examples/tests/test_parametrize_2.py

import pytest

from fibonacci import *

@pytest.mark.parametrize('value1', range(5))
@pytest.mark.parametrize('value2', range(0,10,2))  
def test_methods(value1, value2):
    assert not (value1*value2 & 1)

Overwriting examples/tests/test_parametrize_2.py


In [8]:
! pytest -v examples/tests/test_parametrize_2.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pylint-0.7.1, pep8-1.0.6, cov-2.5.1
collected 25 items                                                              

examples/tests/test_parametrize_2.py::test_methods[0-0] PASSED
examples/tests/test_parametrize_2.py::test_methods[0-1] PASSED
examples/tests/test_parametrize_2.py::test_methods[0-2] PASSED
examples/tests/test_parametrize_2.py::test_methods[0-3] PASSED
examples/tests/test_parametrize_2.py::test_methods[0-4] PASSED
examples/tests/test_parametrize_2.py::test_methods[2-0] PASSED
examples/tests/test_parametrize_2.py::test_methods[2-1] PASSED
examples/tests/test_parametrize_2.py::test_methods[2-2] PASSED
examples/tests/test_parametrize_2.py::test_methods[2-3] PASSED
examples/tests/test_parametri

### `approx`

Il est souvent utile de comparer les valeurs d'un calcul numérique en s'assurant qu'elles sont proches des valeurs attendues.

In [13]:
%%file examples/tests/test_approx_1.py

from pytest import approx

def test_approx_1():
    assert 1.001 == approx(1)
    
def test_approx_2():
    assert 1.001 == approx(1, rel=1e-3)

Overwriting examples/tests/test_approx_1.py


In [14]:
! pytest -v examples/tests/test_approx_1.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pylint-0.7.1, pep8-1.0.6, cov-2.5.1
collected 2 items                                                               

examples/tests/test_approx_1.py::test_approx_1 FAILED
examples/tests/test_approx_1.py::test_approx_2 PASSED

=================================== FAILURES ===================================
________________________________ test_approx_1 _________________________________

    def test_approx_1():
>       assert 1.001 == approx(1)
E       assert 1.001 == 1 ± 1.0e-06
E        +  where 1 ± 1.0e-06 = approx(1)

examples/tests/test_approx_1.py:5: AssertionError
====================== 1 failed, 1 passed in 0.05 seconds ======================


In [27]:
%%file examples/tests/test_approx_2.py

import numpy as np
import pytest
from pytest import approx

def ones_array(shape):
    return np.ones(shape)

@pytest.fixture(params=[5, (3,2), (5, 4, 3)])
def init_array(request):
    return ones_array(request.param)
    
def test_approx(init_array):
    shape = init_array.shape
    random_array = 1 + 1e-5*np.random.random(shape)
    assert random_array == approx(init_array, rel=1e-5)

Overwriting examples/tests/test_approx_2.py


In [28]:
! pytest -v examples/tests/test_approx_2.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pylint-0.7.1, pep8-1.0.6, cov-2.5.1
collected 3 items                                                               

examples/tests/test_approx_2.py::test_approx[init_array0] PASSED
examples/tests/test_approx_2.py::test_approx[5] PASSED
examples/tests/test_approx_2.py::test_approx[init_array2] PASSED

=========================== 3 passed in 0.08 seconds ===========================


### Les `id`

In [30]:
%%file examples/tests/test_approx_id_2.py

import numpy as np
import pytest
from pytest import approx

def ones_array(shape):
    return np.ones(shape)

@pytest.fixture(params=[5, (3,2), (5, 4, 3)], 
                ids=['1d', '2d', '3d'])
def init_array(request):
    return ones_array(request.param)
    
def test_approx(init_array):
    shape = init_array.shape
    random_array = 1 + 1e-5*np.random.random(shape)
    assert random_array == approx(init_array, rel=1e-5)

Writing examples/tests/test_approx_id_2.py


In [31]:
! pytest -v examples/tests/test_approx_id_2.py

============================= test session starts ==============================
platform linux -- Python 3.6.2, pytest-3.2.3, py-1.4.34, pluggy-0.4.0 -- /home/loic/miniconda3/envs/python3.6/bin/python
cachedir: .cache
rootdir: /home/loic/Formations/2017/python/cnrs, inifile:
plugins: pylint-0.7.1, pep8-1.0.6, cov-2.5.1
collected 3 items                                                               

examples/tests/test_approx_id_2.py::test_approx[1d] PASSED
examples/tests/test_approx_id_2.py::test_approx[2d] PASSED
examples/tests/test_approx_id_2.py::test_approx[3d] PASSED

=========================== 3 passed in 0.07 seconds ===========================


### Références

- [La documentation de Pytest](https://docs.pytest.org)
- [Pytest 3.0](https://www.youtube.com/watch?v=HPUg31eylds)
- [Pytest à PyCon2016](https://speakerdeck.com/pycon2016/michael-tom-wing-christie-wilson-introduction-to-unit-testing-in-python-with-pytest)
- [Quatres petits tutos](https://www.youtube.com/watch?v=l32bsaIDoWk&list=PLeo1K3hjS3utzQYDNRNluzqJqpMXx6hHu)

link:

https://speakerdeck.com/pycon2016/michael-tom-wing-christie-wilson-introduction-to-unit-testing-in-python-with-pytest

https://www.youtube.com/watch?v=nznkU7Em5ns

http://il.pycon.org/2016/static/sessions/eli-gur-pytest.pdf

In [2]:
# execute this part to modify the css style\n",
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom.css").read()
    return HTML(styles)
css_styling()